In [1]:
import numpy as np
import pandas as pd
import ast

########################SET folder to path of code directory in the project folder#####################
## SET PATH to code directory in the project folder
code_path = "C:\\Users\\evdoxiataka\\codes\\effi_user_study_2\\code\\"
import sys  
sys.path.insert(1, code_path)

########################SET folder to project directory path#####################
folder= "C:\\Users\\evdoxiataka\\codes\\effi_user_study_2\\"

from utils.data_loading import get_feedback_df, get_training
from utils.utils import attributes_names_mapping
from utils.training import oneoff_training_evaluation, iml_training_evaluation

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

sensitive_attrs = list(attributes_names_mapping.keys())
fs = 0.5

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[inFairness]'
pip install 'aif360[OptimalTransport]'


# Data Loading

In [2]:
## LOAD TRAINING DATA
training_dict = get_training(folder+"data\\collected_data\\LOGS\\","_training.csv") 

In [3]:
## load training, validation and test sets
train_df_train = pd.read_csv(folder+'data\\processed_data\\train.csv', delimiter=',')
train_df_train = train_df_train.loc[:, ~train_df_train.columns.isin(['Unnamed: 0'])]
train_df_test = pd.read_csv(folder+'data\\processed_data\\validation.csv', delimiter=',')
train_df_test = train_df_test.loc[:, ~train_df_test.columns.isin(['Unnamed: 0'])]
test_df = pd.read_csv(folder+'data\\processed_data\\test.csv', delimiter=',') ## test data used to predict labels before shown to participants
test_df = test_df.loc[:, ~test_df.columns.isin(['TARGET','Unnamed: 0'])]

X_train_original = train_df_train.loc[:, ~train_df_train.columns.isin(['TARGET', 'SK_ID_CURR'])]
y_train_original = train_df_train.loc[:, train_df_train.columns == 'TARGET']
X_test_original = train_df_test.loc[:, ~train_df_test.columns.isin(['TARGET', 'SK_ID_CURR'])] ## dataset used for accuracy
y_test_original = train_df_test.loc[:, train_df_test.columns == 'TARGET']

## attributes used in training in correct order
training_attr_in_order = X_train_original.columns.tolist()
training_attr_descriptive_in_order = [attributes_names_mapping[attr[:-3]] if attr.endswith('_LE') else attributes_names_mapping[attr] for attr in training_attr_in_order]

## load original test set to be used for fairness metrics
train_df_test_bin = pd.read_csv(folder+'data\\processed_data\\validation_bin.csv', delimiter=',')
train_df_test_bin = train_df_test_bin.loc[:, ~train_df_test_bin.columns.isin(['Unnamed: 0'])]

## LOAD FEEDBACK
prolific_export_filePath = folder+"data\\collected_data\\prolific_export_demographics.xlsx"
interaction_logs_filePath = folder+"data\\collected_data\\LOGS\\interactions\\"
feedback_df = get_feedback_df(prolific_export_filePath, interaction_logs_filePath)

## discard feedback before revert button click
rev_df = feedback_df[feedback_df["Function"]=="REVERTBUTTON_CLICKED"]
ok_rev_df = feedback_df[feedback_df["Function"].isin(["REVERTBUTTON_CLICKED","OKBUTTON_CLICKED_DECIDE_MODAL_Applications_List"]) ]
rev_idx = rev_df.index.tolist()
ok_rev_idx = ok_rev_df.index.tolist()
ok_rev_idx_idx = [ok_rev_idx.index(i)-1 for i in rev_idx] 
ok_rev_del_idx = [ok_rev_idx[i] for i in ok_rev_idx_idx]
feedback_df = feedback_df[feedback_df['Function']=='OKBUTTON_CLICKED_DECIDE_MODAL_Applications_List']
print(len(feedback_df))
feedback_df = feedback_df[~feedback_df.index.isin(ok_rev_del_idx)]
print(len(feedback_df))
##
feedback_df.drop(columns = "Lower bound",inplace = True)
feedback_df.drop(columns = "Upper bound",inplace = True)

## GET NEW WEIGHTS AND NORMALIZE
all_new_values = []
pred_decision = []
for i in feedback_df.index :
    app_id = feedback_df['App ID'].loc[i]
    p_id = feedback_df['ID'].loc[i]
    value = ast.literal_eval(feedback_df['Value'].loc[i])
    ch_values = {pair['attribute']:float(pair['value']) for pair in value['changed_weights']}
    init_values = {pair['attribute']:float(pair['value']) for pair in ast.literal_eval(value['initial_weights'])}
    # ##
    # tr_idx = 0
    # fd_p_id_app_id = feedback_df[feedback_df['ID']==p_id]
    # fd_p_id_app_id = fd_p_id_app_id[fd_p_id_app_id['App ID']==app_id]
    # if len(fd_p_id_app_id):
    #     tr_idx = fd_p_id_app_id.index.tolist().index(i)
    ## PREDICTED DECISION
    if p_id in training_dict and str(app_id) in training_dict[p_id]:
        # pred_decision.append('Accepted' if training_dict[p_id][str(app_id)][tr_idx]['predicted_decision'][str(app_id)] else 'Rejected')
        pred_decision.append('Accepted' if training_dict[p_id][str(app_id)]['predicted_decision'][str(app_id)] else 'Rejected')
    else:
        pred_decision.append(None)
    ##
    new_value = []
    if len(ch_values)==0:
        all_new_values.append(new_value) 
    else:
        for attr in training_attr_descriptive_in_order:
            if attr in ch_values:
                new_value.append(ch_values[attr])
            else:
                new_value.append(init_values[attr])
        ## NORMALIZE    
        normalizing_factor = sum(new_value)
        new_value = [v/normalizing_factor for v in new_value]
        # if sum(new_value) >1.0:
        #     print(sum(new_value), max(new_value),min(new_value))
        all_new_values.append(new_value)    
feedback_df['Value'] = all_new_values
feedback_df['PredictedDecision'] = pred_decision

23
1544
1533


### Global Model - Labels_Unfair ('unfair' only)
The XGboost classifier is retrained. The feedback instances of all participants are added into the training set all at once.

**Only the 'unfair'** labelled instances by participants are used. 

We set fs = 0.5.

In [ ]:
oneoff_training_evaluation(X_train_original, y_train_original,
                            X_test_original, y_test_original,
                           train_df_test_bin, test_df,
                            sensitive_attrs, fs, feedback_df,
                           True, False,
                           folder+'data\\results\\global\\Labels_Unfair\\','global_Labels_Unfair')

### Global Model - Labels_Unfair+Weights ('unfair')
The XGboost classifier is retrained. The unfair feedback instances of all participants are added into the training set all at once. The attribute weights are set at the average suggested weights from all ('checkes' and 'unfair') feedback instances of participants.

We set fs = 0.5.

In [ ]:
oneoff_training_evaluation(X_train_original, y_train_original,
                            X_test_original, y_test_original,
                           train_df_test_bin, test_df,
                            sensitive_attrs, fs, feedback_df,
                           True, True, folder+'data\\results\\global\\Labels_Unfair+Weights\\','global_Labels_Unfair+Weights')

### Personalized - Labels_Unfair ('unfair')
58 XGboost classifiers will be trained, one for each participant independently. The feedback instances of each participant will be added to the training set incrementally, one at a time, in increasing timestamps (the previous feedback instances of a participant remain in the training set as we add a new one). The XGboost classifier is retrained from scratch in every incremental step (iteration). 

**Only the 'unfair'** labelled instances by participants are used.

We set fs = 0.5.

In [ ]:
iml_training_evaluation(X_train_original, y_train_original,
                        X_test_original, y_test_original,
                        train_df_test_bin, test_df,
                        sensitive_attrs, fs, feedback_df,
                        True, False, 
                        folder+'data\\results\\personalized\\Labels_Unfair\\', 'personalized-Labels_Unfair')

### Personalized - Labels_Unfair-Weights ('unfair')
58 XGboost classifiers will be trained, one for each participant independently. The feedback instances of each participant will be added to the training set incrementally, one at a time, in increasing timestamps (the previous feedback instances of a participant remain in the training set as we add a new one). The XGboost classifier is retrained from scratch in every incremental step (iteration). 

**All the 'unfair'** labelled instances are used integrated in the training set. When an instance is not labeled as unfair but weights are provided the model is retrained using the last training set and setting the weights according to participant's feeeback. Participants weights of each instance are used to set feature weights of the training.

We set fs = 0.5.

In [4]:
iml_training_evaluation(X_train_original, y_train_original,
                        X_test_original, y_test_original,
                        train_df_test_bin, test_df,
                        sensitive_attrs, fs, feedback_df,
                        True, True, 
                        folder+'data\\results\\personalized\\Labels_Unfair+Weights\\','personalized-Labels_Unfair+Weights')

participant 0
participant 1
participant 2
participant 3
participant 4
participant 5
participant 6
participant 7
participant 8
participant 9
participant 10
participant 11
participant 12
participant 13
participant 14
participant 15
participant 16
participant 17
participant 18
participant 19
participant 20
participant 21
participant 22
participant 23
participant 24
participant 25
participant 26
participant 27
participant 28
participant 29
participant 30
participant 31
participant 32
participant 33
participant 34
participant 35
participant 36
participant 37
participant 38
participant 39
participant 40
participant 41
participant 42
participant 43
participant 44
participant 45
participant 46
participant 47
participant 48
participant 49
participant 50
participant 51
participant 52
participant 53
participant 54
participant 55
participant 56
participant 57
participant 58
participant 59
participant 60
participant 61
participant 62
participant 63
participant 64
participant 65
